# Unix Variables

## Safety - Disable default variables

The default behavior is to assign the empty string to undeclared variables. This is a problem because it makes mis-spelled variables hard to detect.

In [ ]:
echo $DOES_NOT_EXIST

In [ ]:
set -u

In [ ]:
echo $DOES_NOT_EXIST

## Assigning variables

In [ ]:
FILENAME="temp.txt"

In [ ]:
echo $FILENAME

In [ ]:
echo "Some stuff" > $FILENAME

In [ ]:
cat $FILENAME

## Common mistakes

You must not have spaces on either side of `=` in a variable assignment.

In [ ]:
FILENAME = "temp.txt"

Unix interprets this as: run a command called `FILENAME`

In [ ]:
FILENAME ="temp.txt"

Unix interprets this as: Assign space to the variable FILENAME then run a program called `temp.txt`

In [ ]:
FILENAME= "temp.txt"

## Using a variable

In [ ]:
PREFIX="Gene"

In [ ]:
echo $PREFIX

In [ ]:
echo $PREFIX001

If you surround the variable name with curly braces, you can concatenate names.

In [ ]:
echo ${PREFIX}001

## Assigning command outputs to variables

To caputre the output of a command, use `$(command)`

In [ ]:
FILES=$(ls)

In [ ]:
echo $FILES

In [ ]:
grep -in "unix" $FILES | head -5

We can also use the anonumous caputre form.

In [ ]:
grep -in "unix" $(ls) | head -5

You may sometimes see this old backticks form. It is equivalent although modern usage seems to favor the `$(command)` from.

In [ ]:
grep -in "unix" `ls` | head -5

## Assigning results of  an arithmetic expression (integers only)

To do integer arithmetic, use `$(( expression ))`.

In [ ]:
echo $((2 + 3))

This does not work for floating point numbers.

In [ ]:
echo $((2.2 + 3.3))

You need to invoke the `bc` calculator program to deal with floating point numbers.

In [ ]:
echo 2.2 + 3.3 | bc 

## Using variables in loops

In [ ]:
for FILE in $(ls *txt)
do
    wc -c $FILE
done

In [ ]:
for FIB in 1 1 2 3 5
do
    echo $FIB
done

In [ ]:
for N in $(seq 1 10)
do
    if [[ $N -le 5 ]]
    then
        echo $N
    else
        echo $((3*N))
    fi
done

## Fibonacci series

Just for fun.

In [ ]:
a=1
b=1
for i in $(seq 1 10)
do
    echo -n ${a}","
    tmp=$a
    a=$b
    b=$((tmp+b))
done

## Single and double quotes

Variables are not evaluated within single quotes, but they are within double quotes.

In [ ]:
FOO=42
echo '$FOO'

In [ ]:
FOO=42
echo "$FOO"

### Testing and branching

Simple example - not use of `-lt`, `-gt`, `&&` and use of parentheses within the test `[[ condition ]]`

In [ ]:
if [[ (2 -gt 1) && (1 -lt 2)]]
then
    echo '2 > 1'
else
    echo 'WTF?'
fi

Check if a file or its uncompressed version exists before downloading.

In [ ]:
URL='ftp://ftp.ensemblgenomes.org/pub/release-39/fungi/gtf/fungi_basidiomycota1_collection/cryptococcus_neoformans_var_grubii_h99/Cryptococcus_neoformans_var_grubii_h99.CNA3.39.gtf.gz'

FILENAME=$(basename $URL)
echo ${FILENAME}
echo ${FILENAME%.*}

# Download if file does not exist
if [[ ! ((-f ${FILENAME}) || (-f ${FILENAME%.*}))  ]]
then
    wget $URL
    gunzip $URL
else
    echo "File exists"
fi

Using regular expression matching in a test.

In [ ]:
for FILE in $(ls)
do
    if [[ $FILE =~ .*Bash.*ipynb$ ]]
    then
        echo $FILE
    fi
done

## Environment variables

You can see what variables are visible in the environment with `env`

In [ ]:
env | head -5

In [ ]:
echo $HOME

To make a variable visible in the general environment so that other programs can use it, you need to `export` it.

In [ ]:
env | grep EXPORTED_VARIABLE

In [ ]:
export EXPORTED_VARIABLE="Hello, Unix"

In [ ]:
env | grep EXPORTED_VARIABLE

Now remove the environment variable.

In [ ]:
unset EXPORTED_VARIABLE

In [ ]:
env | grep EXPORTED_VARIABLE

## Brace expansion

Brace expansions create lists of strings. It can also generate ranges.

In [ ]:
echo file.{c,cpp,py,ipynb,csv,txt}

In [ ]:
echo {a..c}{1..3}.txt

In [ ]:
for NUM in {1..3}; do
    echo mkdir EXPT-${NUM}
done

## Shell scripts

A shell script is just a collection of shell commands that you are now familiar with put into a file that can be executed from the command line. There are a few steps to make a shell script.

1. The first line often contains instructions to use a shell
`#!/bin/bash`
2. The other lines contain standard variable declarations, shell commands, loops etc
3. Save the file with the extension (`.sh`)
4. Make the file executable by `chmod +x <FILENAME>.sh`

Now you can run the shell script as though it were a shell command.

### First shell script

Here we will show the mechanics of creating a shell script.

Use an editor to write `script01.sh`

```bash
#!/bin/bash

echo "Hello bash!"
```

In [ ]:
ls *sh

Change permission to make file executable.

In [ ]:
chmod +x script01.sh

In [ ]:
ls -l *sh

In [ ]:
./script01.sh

### Second shell script

Here we see how to pass arguments to a script in `script02.sh`

```bash
#!/bin/bash                                                                     

echo '$# gives $ of arguments     :' $#
echo '$@ gives arguments as array :' $@
echo '$* gives arguments as string:' $*

echo '$1, $2, $3 give firs, second, third arguments etc'
echo '$1:' $1
echo '$2:' $2
echo '$2:' $3

echo 'Evaluating "$@"'
for ARG in "$@"
do
    echo ${ARG}
done
echo 'Evaluating "$*"'
for ARG in $*
do
    echo ${ARG}
done


```

In [ ]:
chmod +x script02.sh

In [ ]:
./script02.sh a b "c d"

## Exercises

1. Write a shell script that accepts an arbitrary number of filenames as arguments (possibly given by `ls`), and outputs the total number of words in those files.

```bash
#!/bin/bash

TOTAL=0
for FILE in "$@"
do
    N=$(wc -w < $FILE)
    TOTAL=$((TOTAL + N))
done
echo $TOTAL
```